In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
# pretrained_model = "distilbert-base-uncased"
# pretrained_model = "xlm-roberta-large"
pretrained_model = "microsoft/deberta-v3-small"
pretrained_model = "microsoft/deberta-v3-base"
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "conll2003"
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}"

use_pretrained_model = True
if use_pretrained_model:
    model_checkpoint = pretrained_model

batch_size = 16
num_train_epochs = 6

# dataset_name = "wnut_17"
dataset_name = "conll2003"

In [2]:
import neptune.new as neptune
from transformers.integrations import NeptuneCallback

run_name = f'{pretrained_model}' if use_pretrained_model else f'{pretrained_model}/{source_dataset_name}'
# Create a Neptune run object
run = neptune.init_run(
    project="thingumajig/ner-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhN2QyZGRkNS0yNDhhLTQyYWItOTg5ZC1kZGQ3Yjc3ZDMxZTcifQ==",
    
    name=f"train-{run_name}:{dataset_name}", description=f"{run_name} fine-tuned on {dataset_name}",
    
    tags=[pretrained_model, dataset_name, "NER"],  # 

    source_files=["*.ipynb"],  # 

)

neptune_callback = NeptuneCallback(run=run)


2022-10-09 07:16:15.373418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 07:16:15.477210: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-09 07:16:15.838988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-09 07:16:15.839033: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-2
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Loading dataset

In [3]:
from datasets import load_dataset, load_metric

In [4]:
datasets = load_dataset(dataset_name)

Found cached dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,731,"[8, -, Jim, Courier, (, U.S., ), vs., Javier, Sanchez, (, Spain, )]","[CD, :, NNP, NNP, (, NNP, ), IN, NNP, NNP, (, NNP, )]","[B-NP, I-NP, I-NP, I-NP, O, B-NP, O, B-PP, B-NP, I-NP, O, B-NP, O]","[O, O, B-PER, I-PER, O, B-LOC, O, O, B-PER, I-PER, O, B-LOC, O]"
1,1534,"[We, will, be, guests, of, the, Kennedys, ,, "", said, Frank, Quilter, ,, one, of, the, two, who, have, been, invited, to, Chicago, .]","[PRP, MD, VB, NNS, IN, DT, NNPS, ,, "", VBD, NNP, NNP, ,, CD, IN, DT, CD, WP, VBP, VBN, VBN, TO, NNP, .]","[B-NP, B-VP, I-VP, B-NP, B-PP, B-NP, I-NP, O, O, B-VP, B-NP, I-NP, O, B-NP, B-PP, B-NP, I-NP, B-NP, B-VP, I-VP, I-VP, B-PP, B-NP, O]","[O, O, O, O, O, O, B-PER, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, B-LOC, O]"
2,1425,"[I, dont, know, if, Alicia, is, working, out, .]","[PRP, VBP, VB, IN, NNP, VBZ, VBG, RP, .]","[B-NP, B-VP, I-VP, B-SBAR, B-NP, B-VP, I-VP, B-PRT, O]","[O, O, O, O, B-PER, O, O, O, O]"
3,3226,"[RTRS, -, Niugini, shares, surge, on, bid, talk, .]","[NNS, :, NNP, NNS, NN, IN, NN, NN, .]","[B-NP, O, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[B-ORG, O, B-ORG, O, O, O, O, O, O]"
4,12717,"[U.S., EMBASSY, IN, ATHENS, CLOSED, ON, LABOUR, DAY, ,, SEP, 2, .]","[NNP, NNP, IN, VBZ, VBN, IN, NNP, NNP, ,, NN, CD, .]","[B-NP, I-NP, B-PP, B-VP, I-VP, B-PP, B-NP, I-NP, O, B-NP, I-NP, O]","[B-LOC, B-MISC, O, B-LOC, O, O, B-MISC, I-MISC, O, O, O, O]"
5,4933,"[Standings, (, (, tabulated, under, played, ,, won, ,, drawn, ,, lost, ,, goals]","[NNS, (, (, VBN, IN, JJ, ,, VBD, ,, VBN, ,, VBD, ,, NNS]","[B-NP, O, O, B-VP, B-PP, B-NP, O, B-VP, B-VP, I-VP, O, B-VP, O, B-NP]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,11453,[1996-08-28],[CD],[B-NP],[O]
7,13598,"[The, pensioner, ,, named, only, as, Pietro, T., ,, told, investigators, he, was, infected, with, HIV, ,, the, AIDS, virus, ,, and, his, wife, had, died, of, the, disease, .]","[DT, NN, ,, VBN, RB, IN, NNP, NNP, ,, VBD, NNS, PRP, VBD, JJ, IN, NNP, ,, DT, NNP, NN, ,, CC, PRP$, NN, VBD, VBN, IN, DT, NN, .]","[B-NP, I-NP, O, B-VP, B-ADVP, B-PP, B-NP, I-NP, O, B-VP, B-NP, B-NP, B-VP, B-NP, B-PP, B-NP, O, B-NP, I-NP, I-NP, O, O, B-NP, I-NP, B-VP, I-VP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, B-MISC, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O]"
8,9486,"[Afghan, guerrilla, factions, have, been, locked, in, a, bloody, power, struggle, since, the, fall, of, the, communist, government, in, April, 1992, .]","[JJ, NN, NNS, VBP, VBN, VBN, IN, DT, JJ, NN, NN, IN, DT, NN, IN, DT, NN, NN, IN, NNP, CD, .]","[B-NP, I-NP, I-NP, B-VP, I-VP, I-VP, B-PP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,3838,"[MINNESOTA, 64, 64, .500, 12]","[NNP, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O]"


# Preprocessing dataset

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [15]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁Germany', "▁'", 's', '▁representative', '▁to', '▁the', '▁European', '▁Union', "▁'", 's', '▁veterinary', '▁committee', '▁Werner', '▁Z', 'wing', 'mann', '▁said', '▁on', '▁Wednesday', '▁consumers', '▁should', '▁buy', '▁sheep', 'meat', '▁from', '▁countries', '▁other', '▁than', '▁Britain', '▁until', '▁the', '▁scientific', '▁advice', '▁was', '▁clearer', '▁.', '[SEP]']


In [16]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 38)

In [17]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [18]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

38 38


In [19]:
label_all_tokens = True

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 2805, 27144, 2324, 660, 264, 20007, 1668, 12649, 323, 2], [1, 2557, 28080, 2], [1, 98641, 4773, 271, 4425, 271, 2944, 2], [1, 279, 1559, 2653, 357, 277, 1561, 278, 26748, 275, 2324, 1678, 264, 2888, 264, 48371, 1668, 12649, 583, 3959, 2003, 786, 6943, 9925, 1682, 295, 282, 10364, 264, 8260, 323, 2], [1, 2439, 382, 268, 4290, 264, 262, 1559, 2432, 382, 268, 15275, 3066, 31127, 2548, 7562, 8556, 357, 277, 1623, 2888, 403, 809, 8260, 34539, 292, 1226, 340, 354, 3491, 583, 262, 3122, 1678, 284, 14025, 323, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-db6dcf5600e9ae38.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-421747f9413facda.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-4471bc31f5f9040c.arrow


# Fine-tuning the model

In [23]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForTokenClassification: ['deberta.embeddings.position_embeddings.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Be

In [24]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    
    save_strategy='epoch',
    save_total_limit = 3,
    
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    
    report_to="none",
)

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

/tmp/ipykernel_2390198/1092872775.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [27]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [28]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Trainer

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
    callbacks=[neptune_callback],
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5268
You're using a DebertaV2TokenizerFast tokenizer.

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.168500,0.048684,0.937712,0.946527,0.942099,0.988157
2,0.036000,0.038996,0.949083,0.960322,0.954670,0.990805
3,0.019400,0.038016,0.957127,0.964839,0.960968,0.991907
4,0.013200,0.038791,0.955975,0.964962,0.960447,0.991874
5,0.008500,0.040325,0.960753,0.965328,0.963035,0.992138
6,0.005000,0.040933,0.959748,0.966427,0.963076,0.992253


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to ../data/models/microsoft/deberta-v3-base/conll2003/checkpoint-878
Configuration saved in ../data/models/microsoft/deberta-v3-base/conll2003/checkpoint-878/config.json
Model weights saved in ../data/models/microsoft/deberta-v3-base/conll2003/checkpoint-878/pytorch_model.bin
tokenizer config file saved in ../data/models/microsoft/deberta-v3-base/conll2003/checkpoint-878/tokenizer_config.json
Special tokens file saved in ../data/models/microsoft/deberta-v3-base/conll2003/checkpoint-878/special_tokens_map.json
/root/.cache/pypoetry/virtualenvs/sequence-labeling-

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 16 operations to synchronize with Neptune. Do not kill this process.
All 16 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-2


TrainOutput(global_step=5268, training_loss=0.03369786482284534, metrics={'train_runtime': 446.2181, 'train_samples_per_second': 188.8, 'train_steps_per_second': 11.806, 'total_flos': 1954364746278528.0, 'train_loss': 0.03369786482284534, 'epoch': 6.0})

In [32]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829

parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.


NeptuneMissingConfiguration: 
        ------ Unsupported ---- We were not able to create new runs. You provided a custom Neptune run to
        `NeptuneCallback` with the `run` argument. For the integration to work fully, provide your `api_token` and
        `project` by saving them as environment variables or passing them to the callback.
        

In [33]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3250
  Batch size = 16


{'LOC': {'precision': 0.9789755351681957,
  'recall': 0.9797245600612089,
  'f1': 0.9793499043977054,
  'number': 2614},
 'MISC': {'precision': 0.912621359223301,
  'recall': 0.9102112676056338,
  'f1': 0.9114147201410313,
  'number': 1136},
 'ORG': {'precision': 0.9379084967320261,
  'recall': 0.9587973273942093,
  'f1': 0.9482378854625549,
  'number': 1796},
 'PER': {'precision': 0.9759669545625235,
  'recall': 0.9826086956521739,
  'f1': 0.979276563677468,
  'number': 2645},
 'overall_precision': 0.9597478176527643,
 'overall_recall': 0.9664265657428885,
 'overall_f1': 0.9630756128718292,
 'overall_accuracy': 0.992252779788144}

In [ ]:
trainer.log(results)

In [47]:
run.stop()
